# Draw a Brick Pattern

Attempt at programmatically making a brick pattern.

All units in mm. ```1``` = ```1 mm```.

"Napkin" scratches.

![](http://luckofthedraw.fun/.imgs/brick_0003.jpeg)

Drawn by hand. ~18mm brick height.

![](http://luckofthedraw.fun/.imgs/brick_0001.jpeg)

In [1]:
import GCode
import GRBL
import numpy as np
import os
import sys
from time import sleep


> Standard bricks. The standard co-ordinating size for brickwork is 225 mm x 112.5 mm x 75 mm (length x depth x height). This includes 10 mm mortar joints, and so the standard size for a brick itself is 215 mm x 102.5 mm x 65 mm (length x depth x height).

In [3]:
# Standard brick dimensions.
BrickHeight = 65  # [mm]
BrickLength = 225  # [mm]
BrickDepth = 12.5  # [mm]
BrickRatio = 215 / 65  # [dimensionless]


In [4]:
# Poplar 1x4". Cut
BlockHeight = 89.0  # mm
BlockLength = 2 * BlockHeight  # mm


In [5]:
# Drawing configuration.
# How many rows of bricks to draw on the block.
N_BrickRows = 5  # [dimensionless]

# Dimensions of a 'brick' projected onto the block of wood.
H_Block_Brick = BlockHeight / N_BrickRows  # [mm]
L_Block_Brick = H_Block_Brick * BrickRatio, 4  # [mm]


# Code:

# Default Line

Sane default for the ```draw_line``` command. Designed so that ```draw_line()``` does something that is easly measureable.

Draw a 30-60-90 triangle.

From: https://www.dummies.com/education/math/calculus/how-to-work-with-30-60-90-degree-triangles.

If you look at the 30:60:90-degree triangle in radians, it translates to the following:

$$\frac{\pi}{6}:\frac{\pi}{3}:\frac{\pi}{2}$$

In any 30-60-90 triangle, you see the following:

- The shortest leg is across from the 30-degree angle.

- The length of the hypotenuse is always two times the length of the shortest leg.

- You can find the long leg by multiplying the short leg by the square root of 3.

If you know one side of a 30-60-90 triangle, you can find the other two by using shortcuts. Here are the three situations you come across when doing these calculations:

- **Type 1**: You know the short leg (the side across from the 30-degree angle). Double its length to find the hypotenuse. You can multiply the short side by the square root of 3 to find the long leg.

- **Type 2**: You know the hypotenuse. Divide the hypotenuse by 2 to find the short side. Multiply this answer by the square root of 3 to find the long leg.

- **Type 3**: You know the long leg (the side across from the 60-degree angle). Divide this side by the square root of 3 to find the short side. Double that figure to find the hypotenuse.

Let:

- a: Shortest Side. Opposite 30$^o$ ($\frac{\pi}{6}$)

In [16]:
a = 10  # [mm]. Shorest leg of the triangle will be 10 mm, 1 cm, 0.01 m long.


In [18]:
default_points = np.array(
    [
        [0, 0],  # Start at origin.
        [a * np.sqrt(3), 0],  # Draw long side along X axis.
        [a * np.sqrt(3), a],  # Draw the short side parallel to Y axis.
        [0, 0],  # Return to origin. Draw hypotenuse.
    ]
)
default_points


array([[ 0.        ,  0.        ],
       [17.32050808,  0.        ],
       [17.32050808, 10.        ],
       [ 0.        ,  0.        ]])

Default settings for feed rate and laser power:

In [19]:
default_feed = 300  # mm/n
default_power = 150  # [dimensionless]


In [40]:
class GCode(object):
    NEWLINE = "\n"
    def __init__(self, file=None, machine=None, buffer=None):
        self.machine = machine
        if file is not None:
            self.load(file)

        if buffer is None:
            self.buffer = list()
        else:
            self.buffer = buffer

    @property
    def code(self):
        return GCode.NEWLINE.join(self.buffer)

    def load(self, filename):
        with open(filename, "r") as fid:
            data = fid.read()
        self.buffer = data.splitlines()

    def save(self, filename):
        with open(filename, "w") as fid:
            print(str(self), file=fid)

    def __str__(self):
        return self.code

    def __repr__(self):
        return "<GCode>[cmds={}]".format(len(self.buffer))

    def _repr_html_(self):
        html = list()
        for cmd_line in self.buffer:
            cmd, *args = cmd_line.split(" ")
            html_line = "<b>{cmd}</b> <i>{args}</i>".format(
                cmd=cmd, args=" ".join(args)
            )
            html.append(html_line)
        return "<br>\n".join(html)

    def __add__(self, other):
        buffer = self.buffer
        buffer2 = other.buffer

        buffer.extend(buffer2)

        return GCode(machine=self.machine, buffer=buffer)

    def __iter__(self):
        """ __iter__ function """
        for i in range(len(self.buffer)):
            yield(self.buffer[i])


    def run(self):
        """ run the program on the given machine """
        if self.machine is None:
            raise Exception("No machine to run on")
        self.machine.run(self)

    def optimise(self):
        """ Create the best GCode possible. """
        raise(NotImplementedError("TODO:"))

numeric_types = (int, int8, int16, int32, int64, np.float, np.float16, np.float32, np.float64, np.float128)

def cmd_factory(cmd, doc=None):
    """ Factory to create GCode Command Functions. """
        
    def cmd_fcn(self, **kwargs):
        args = list()
        
        
        for key, value in kwargs.items():
            value_type is 
            
            args.append("{key}{value}".format(key=key, value=value))

        cmd_str = "{cmd} {args}".format(cmd=cmd, args=" ".join(args))
        # For commands with no arguments.
        cmd_str = cmd_str.strip()
        self.buffer.append(cmd_str)

    return cmd_fcn


# Good core to start with.
commands = list()
# GCodes
for code in [0, 1, 2, 3, 4, 20, 21, 28, 90, 91, 92]:
    commands.append("G{code}".format(code=code))
# MCodes
for code in [0, 1, 2, 3, 4, 5, 6]:
    commands.append("M{code}".format(code=code))

for command in commands:
    setattr(GCode, command, cmd_factory(command))

class Line(GCode):
    def __init__(
        self,
        points=default_points,
        feed=default_feed,
        power=default_power,
        dynamic_power=True,
        *args,
        **kwargs,
    ):
        super().__init__(
            *args,
            **kwargs,
        )
        
        self.points = points
        self.feed = feed
        self.power = power
        self.dynamic_power = dynamic_power
        
    def generate_gcode(self):
        self.buffer = list()
        self.G0(F=60) # 60 mm / min = 1 mm / sec
        self.G1(F=60) # 60 mm / min = 1 mm / sec
        self.M3(S=1) # Set laser power so that movement can be seen, but does nothing.
        self.G28() # "Home"
        self.G21() # Metric Units
        self.G90() # Absolute positioning.
        self.G92(X=0, Y=0, Z=0) # The cliche, I forgot why I added it. It works. Don't touch it.
        
        
        self.G0(
            X=self.points[0,0],
            Y=self.points[0,1]
        )

self = Line()
self.G0(X=0, Y=0)
self.G0(X=10, Y=5)

self.generate_gcode()

self.buffer

SyntaxError: invalid syntax (<ipython-input-40-b0105c89ccc5>, line 76)

In [36]:
self.optimise()


NotImplementedError: TODO:

In [37]:
for x in self:
    print(type(x))
    print(x)
    print()


__iter__
<class 'str'>
G0 F60

<class 'str'>
G1 F60

<class 'str'>
M3 S1

<class 'str'>
G28

<class 'str'>
G21

<class 'str'>
G90

<class 'str'>
G92 X0 Y0 Z0

<class 'str'>
G0 X0.0 Y0.0

